This is the code that will be imported into our Raspberry Pi to process and send images to our computer for classification

In [24]:
# Import packages

# reload our external packages (tracker)
import importlib

import numpy as np
import cv2
import os
from skimage import data, filters
import tracker as track
import threading  # Import the threading module

importlib.reload(track)

# Directory to save training images

# base_dir = "/Users/williamlee/Documents/Git Repos/Lego-Brick-Sorter/Imaging Pipeline/testImages"
# base_dir = "/home/billiam/Documents/Lego_Sorter/TestPipelineImages"
base_dir = "/home/billiam/Documents/Lego_Sorter/CREATED BATCHES"

os.chdir(base_dir)
print(os.listdir())

[]


In [ ]:
# pieceNum = "3021"
# os.chdir(pieceNum)
# Open Video
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(cv2.CAP_V4L2)

# Adjust Camera Settings
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1) # Manual exposure mode
cap.set(cv2.CAP_PROP_EXPOSURE, 40)
cap.set(cv2.CAP_PROP_AUTO_WB, 0) # Disable automatic white balance (Not sure if this is working)

# Create Tracker Object
tracker = track.EuclideanDistTracker()

# Create BG Subtraction Function with set parameters
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
fgbg.setVarThreshold(100) #The threshold of what is detected as not background (150 can detect regular pieces, but not white)

# Create Window
cv2.startWindowThread()

# Function to save an image using cv2.imwrite in a separate thread
def save_image_thread(roi, uniqueId, age):
    save_path = os.path.join(base_dir, f"{uniqueId}_{age}.png")
    cv2.imwrite(save_path, roi)
    
# Set up the gamma correction factor
gamma = 1.2  # Adjust this value to control the contrast (1.0 means no change)

# Function to apply gamma correction to an image
def adjust_gamma(image, gamma=1.0):
    # Build a lookup table mapping the pixel values [0, 255] to their adjusted gamma values
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype(np.uint8)
    # Apply the gamma correction using the lookup table
    return cv2.LUT(image, table)

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    
    # 0: Pre-Process Image
    window_factor = 1      # Size factor of display window
    x_left_crop = 0        # Left side window crop
    x_right_crop = 1920     # Right side window crop
    y_top_crop = 0          # Top side window crop
    y_bottom_crop = 1080    # Bottom side window crop
    saturation_factor = 1.2 # Factor to saturate image by
    value_factor = 1.2      # Factor to increase value of image by

    # Crop and adjust saturation of image. Blur to reduce noise from saturation
    #frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)
    #frame = cv2.GaussianBlur(frame, (25, 25), 0)
    
    # Apply gamma correction
    frame = adjust_gamma(frame, gamma=gamma)
    
    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Adjust size
    #frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    #fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)

    
    # 1: Object Detection
    min_area = 300 # Min area of contour to detect as piece
    
    # Iterate through discovered contours and add detections
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    image_copy = frame.copy()
    contourFrame = frame.copy()
    
    cv2.drawContours(contourFrame, contours, -1, (0,255,0), 3)
    
    for i, contour in enumerate(contours):
        
        if hierarchy[0,i,3] == -1:
            x, y, w, h = cv2.boundingRect(contour)
            # If contour overlap with border then skip
            if((x == 0) or (y == 0) or (x+w >= frame.shape[1]) or (y+h >= frame.shape[0])):
                continue
            
            area = cv2.contourArea(contour)
            if(area > min_area):
                cv2.putText(contourFrame, "Area: " + str(area), (x, y-10), 0,0.5, (255,0,0), 2)
                detections.append([x, y, w, h])
                
    # In order to detect white pieces, we need to write a function that takes all the contours, looks at the ones that make the size criteria, and then
    # finds all nearby contours and adds them to the contour space. The white pieces being detected as a bunch of small contours
    
    # 2: Object Tracking
    min_age = 30 # Min age of object to classify as brick
    interval = 1 # Frequency of images to save after being classified as brick
    
    box_ids = tracker.update(detections)
    for box_id in box_ids:
        x, y, w, h, id, age, uniqueId = box_id
        # If a box is "old enough" box is determined to be a piece --> take pictures
        if age > min_age:
            larger = w
            if(h > w):
                larger = h
            cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
            cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(age) + " id:" + str(uniqueId),(x,y-10),0,0.5,(255, 0, 0), 2)
            
            # Save bounding box as image every X frames
            if x and y > 10:
                padding = 10
            else:
                padding = 0
            roi = frame[y:y+larger, x:x+larger]
            true_age = age - min_age
            
            if(true_age % interval == 0):
                print(true_age)
                print("save image")
                #cv2.imwrite(str(uniqueId) + "_" + str(age) + ".png", roi)
                
                # Start a new thread to save the image
                save_thread = threading.Thread(target=save_image_thread, args=(roi.copy(), uniqueId, age))
                save_thread.start()
            
    
    cv2.imshow('Feed with bounding boxes', image_copy)
    #cv2.imshow('MOG2 Output', fgmask)
    cv2.imshow('Contours', contourFrame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

1
save image
2
save image
3
save image
4
save image
5
save image
6
save image
7
save image
8
save image
9
save image
10
save image
11
save image
12
save image
13
save image
14
save image
15
save image
16
save image
17
save image
18
save image
19
save image
20
save image
21
save image
22
save image
23
save image
24
save image
25
save image
26
save image
27
save image
28
save image
29
save image
30
save image
31
save image
32
save image
33
save image
34
save image
35
save image
36
save image
37
save image
38
save image
39
save image
40
save image
41
save image
42
save image
1
save image
2
save image
3
save image
4
save image
5
save image
6
save image
7
save image
8
save image
9
save image
10
save image
11
save image
12
save image
13
save image
14
save image
15
save image
16
save image
17
save image
18
save image
1
save image
2
save image
3
save image
4
save image
5
save image
6
save image
7
save image
8
save image
9
save image
10
save image
11
save image
12
save image
13
save image
14
sa